In [16]:
import helpers as h

import numpy as np
import pandas as pd
from scipy.optimize import minimize, NonlinearConstraint

from icecream import ic

In [2]:
# example 
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571


In [3]:
# constants 
# states_path = "data/states/nga_admbnda_adm1_osgof_20161215.shp"
# states = gpd.read_file(states_path)

data_path_names = ["wind_speed", "temperature", "solar_flux"]

data_averages = {}
for name in data_path_names:
    data_averages[name] = pd.read_csv(f"clean_data/{name}.csv", index_col=0)

avg_vals = {n: i["value"] for n, i in data_averages.items()}

In [4]:
state_areas = pd.read_csv(f"clean_data/state_areas.csv")

In [5]:
def get_constants():
    data_path_names = ["wind_speed", "temperature", "solar_flux"]

    data_averages = {}
    for name in data_path_names:
        data_averages[name] = pd.read_csv(f"clean_data/{name}.csv", index_col=0)

    avg_vals = {n: i["value"] for n, i in data_averages.items()}

    state_areas = pd.read_csv(f"clean_data/state_areas.csv")

    F_cur=avg_vals["solar_flux"]
    T_a=avg_vals["temperature"]
    w=avg_vals["wind_speed"]

    return (F_cur, T_a, w, state_areas)

    

In [6]:
# F_cur, T_a, w, state_areas = get_constants()


In [7]:
def do_calc(x, constants):
    """ 
    - pl_solar, pl_wind ~ percent of land 
    - ns_solar, ns_wind ~ number of states 
    """

    pl_solar, pl_wind, ns_solar, ns_wind = x
    # ic(pl_solar, pl_wind, ns_solar, ns_wind )
    

    # constant values 
    F_cur, T_a, w, state_areas = constants[0]
    
    # solar capacity for each state 
    P_ac = h.calculate_power_panel(F_cur, T_a, w)

    # return P_ac

    # wind capacity for each state 
    P_m, P_t, CF, P_t_after_losses, P_r = h.calculate_power_turbine(V_m=w)

    # panel distribution 
    solar_res = h.calc_num_panels(pl_solar, 
        ns_wind, 
        state_areas, 
        P_ac)

    # turbine distribution 
    wind_res = h.calc_num_turbines(pl_wind, 
        ns_wind, 
        state_areas, 
        P_t_after_losses, 
        P_r)

    return solar_res, wind_res

In [8]:
int(4.0)

4

In [35]:
class optimize_energy():
    def __init__(self):
        c = get_constants()
        self.F_cur = c[0]
        self.T_a = c[1]
        self.w = c[2] 
        self.state_areas = c[3]

    def do_calc(self, x):
        """ 
        - pl_solar, pl_wind ~ percent of land 
        - ns_solar, ns_wind ~ number of states 
        """
        pl_solar, pl_wind, ns_solar, ns_wind = x

        # capacities for each state 
        P_ac = h.calculate_power_panel(self.F_cur, self.T_a, self.w)
        P_m, P_t, CF, P_t_after_losses, P_r = h.calculate_power_turbine(V_m=self.w)

        # distributions
        solar_res = h.calc_num_panels(pl_solar, 
            ns_wind, 
            self.state_areas, 
            P_ac)
        wind_res = h.calc_num_turbines(pl_wind, 
            ns_wind, 
            self.state_areas, 
            P_t_after_losses, 
            P_r)

        return solar_res, wind_res

    def min_devices(self, x):
        solar_res, wind_res = self.do_calc(x)
        num_devices = solar_res["n_panels"].sum() + wind_res["n_turbines"].sum()

        return num_devices

    def meet_energy(self, x):
        solar_res, wind_res = self.do_calc(x)
        self.total_energy = solar_res["total_energy (mwh)"] + wind_res["total_energy (mwh)"]
        self.energy_need = 26_5337_353 # 27 million MWh 
        self.dif = total_energy - energy_need # > 0 

        return dif 
    

In [33]:
o = optimize_energy()

In [30]:
x0 = [0.02, 0.02, 4, 4]

o.meet_energy(x0)

TypeError: meet_energy() takes 1 positional argument but 2 were given

In [36]:
# constants = get_constants()
o = optimize_energy()
x0 = [0.02, 0.02, 4, 4]

nlc  = NonlinearConstraint(fun=o.meet_energy, lb=0, ub=1e3)
res = minimize(fun=o.min_devices, x0=x0, method='nelder-mead', constraints=nlc,
               options={'maxiter': 10, 'disp': True})

/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/optimize/_minimize.py:565: RuntimeWarning: Method nelder-mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,
/var/folders/7r/39t1cgb13d57zk26c__2z50m0000gn/T/ipykernel_11559/3144562946.py:6: RuntimeWarning: Maximum number of iterations has been exceeded.
  res = minimize(fun=o.min_devices, x0=x0, method='nelder-mead', constraints=nlc,


In [18]:
def min_devices(x, constants):
    solar_res, wind_res = do_calc(x, constants)
    num_devices = solar_res["n_panels"].sum() + wind_res["n_turbines"].sum()

    return num_devices

def meet_energy(x,):
    constants = get_constants()
    solar_res, wind_res = do_calc(x, constants)
    total_energy = solar_res["total_energy (mwh)"] + wind_res["total_energy (mwh)"]
    energy_need = 26_5337_353 # 27 million MWh 
    dif = total_energy - energy_need # > 0 

    return dif 


In [10]:
constants = get_constants()
x0 = [0.02, 0.02, 4, 4]
a = do_calc(x0, [constants])
# a.sort_values(ascending=False)[0:3]

In [37]:
print(res.x)
meet_energy(res.x)

[0.01193164 0.02189868 5.13540039 3.49594727]


TypeError: meet_energy() takes 1 positional argument but 2 were given

array([0.01193164, 0.02189868, 5.13540039, 3.49594727])

- things to minimize potentially
  - total land used 
  - number of turbines + number of panels  ~ cost 
- start by definining seperate function 

- [mnimize where output has contraints](https://stackoverflow.com/questions/64169852/optimizing-input-where-output-has-constraints-in-python)
- 
- [use a class for constants](https://stackoverflow.com/questions/64205381/can-i-send-arguments-to-a-constraint-function-using-scipy-optimize-nonlinearcons)

In [ ]:
# things to minimize potentially 
    